# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [38]:
#define bootstrap functions (from datacamp)

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""
    
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [84]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='b'].call))
print(len(data[data.race=='b'].call))

print(sum(data[data.race=='w'].call))
print(len(data[data.race=='b'].call))


157.0
2435
235.0
2435


The rate of callbacks are 157/2435 and 235/2435 for blacks and whites respectively.

In [19]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


#### Question 1:
- It seems a 2 sample test would be appropriate, as the resumes are made to be as similar as possible except for the name.  The Central Limit Theory should apply, as the sample is large enough.

#### Question 2:

- The null hypothesis is that the difference in callback rates between applications with black sounding names and those with white sounding names is not statistically significant.

- The alternate hypotheses is that using a white or black sounding name on a resume does affect the number of callbacks.

In [106]:
w = data[data.race=='w']
b = data[data.race=='b']

In [97]:
#overall call rate

x = sum(data.call == 1)
print('Number of Callbacks: ', x)
y = len(data.call)
print('Number of Applications: ',y)
call_rate = x/y
print('Average Call Rate: ', round(call_rate,5))


#call rate for black sounding names
bx = sum(b.call ==1)
by = len(b.call)
b_rate = (bx/by)
print('Callback rate for black sounding names', round(b_rate,5))

#call rate for white sounding names
wx = sum(w.call ==1)
wy = len(w.call)
w_rate = (wx/wy)
print('Callback rate for white sounding names:', round(w_rate,5))

b_diff = w_rate - b_rate
print('Difference between callback rates of black and whites:', round(b_diff,5))

Number of Callbacks:  392
Number of Applications:  4870
Average Call Rate:  0.08049
Callback rate for black sounding names 0.06448
Callback rate for white sounding names: 0.09651
Difference between callback rates of black and whites: 0.03203


The average callback rate overall is 392 out 4870. which is about 8%. The callback rate for white sounding names was around 9.6% and the rate for black sounding names was 6.4%, or 1.6% lower than the expected norm and 32 % away from those with white sounding names.

In [98]:
#shift the data for comparison

b_shifted = b.call - np.mean(b.call) + np.mean(w.call)


In [99]:
# test the data on black sounding names through 10,000 repetitions

boot = draw_bs_reps(b_shifted, np.mean, 10000)
print(boot.mean())
print(boot.min())

0.09660628407821059
0.0796714574098587


In [100]:
# Your solution to Q3 here

# calculate mean, 95% confidence interval, margin of error


print ('Call Rate:', boot.mean())
print ('Standard Deviation: ', np.std(boot))
print ('95% Confidence Interval:', np.percentile(boot, [2.5, 97.5]))
print ('Margin of Error:', 1.96 * np.std(boot)/ np.sqrt(2435))


Call Rate: 0.09660628407821059
Standard Deviation:  0.004983963597473156
95% Confidence Interval: [0.08665298 0.10636552]
Margin of Error: 0.00019796182558825946


In [107]:
import scipy
from scipy import stats

w_difference =scipy.stats.ttest_ind(b.call, w.call)
print(w_difference)

Ttest_indResult(statistic=-4.114705290861751, pvalue=3.940802103128886e-05)



###### 4. Write a story describing the statistical significance in the context or the original problem.
    
Judging by the very small p-value which is significantly below the the .05 threshold for statistical significance,  it seems that the null hypothesis, that black sounding names received fewer callbacks based on random chance alone is an unlikely scenario.</p>


##### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis does not indicate that race is the most important factor in callbacks, it merely suggests that the difference in callback rate is statistically significant, or likely not to be the result of random chance.

The study could be ammended to include resumes with similar sounding names but differences in other variables to explore whether or not these are equally impactful.